In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [2]:

img_height = 150
img_width = 150
batch_size = 32
data_directory = '/Users/baudi/AI/practicas/uvas/data/train_val/'
test_data_directory = '/Users/baudi/AI/practicas/uvas/data/test/'
val_split = 0.2
seed = 42
num_classes = 4
learning_rate = 0.0001


In [3]:

def adjustments(image):
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    return image

def process_labels(images, labels):
    one_hot_labels = tf.one_hot(labels, depth=num_classes)
    return images, one_hot_labels

train_data_raw = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_data_raw.class_names

train_data = train_data_raw.map(lambda x, y: (adjustments(x), y)).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


val_data = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)

test_data = image_dataset_from_directory(
    test_data_directory,
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


Found 3248 files belonging to 4 classes.
Using 2599 files for training.
Metal device set to: Apple M2 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Found 3248 files belonging to 4 classes.
Using 649 files for validation.
Found 814 files belonging to 4 classes.


2023-04-10 12:05:07.958054: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-10 12:05:07.958224: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
def build_model():

    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(1024, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(2048, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 clases: sanas, enfermedad1, enfermedad2, enfermedad3
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model


In [5]:

def cross_validate(model_builder, n_splits=2, epochs=20):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    accuracies = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y), start=1):
        model = model_builder()
        print(f"Comienza el entrenamiento del fold {fold}")
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

        callbacks = [early_stopping, reduce_lr]

        history = model.fit(X[train_idx],
                  y[train_idx],
                  validation_data=(X[val_idx], y[val_idx]),
                  batch_size = batch_size,
                  epochs=epochs,
                  callbacks=callbacks)
        print(f"Termina el entrenamiento del fold {fold}")
        val_accuracy = history.history['val_accuracy']
        last_val_accuracy = val_accuracy[-1]
        print(f"Precisión de validación del fold {fold}: {last_val_accuracy}")

        accuracies.append(last_val_accuracy)

    return np.mean(accuracies)


In [11]:

mean_accuracy = cross_validate(build_model,5,50)
print(f'Mean accuracy: {mean_accuracy:.4f}')


Comienza el entrenamiento del fold 1
Epoch 1/50


2023-04-10 12:37:05.654203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - ETA: 0s - loss: 0.7140 - accuracy: 0.7948

2023-04-10 12:37:14.722215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - 10s 116ms/step - loss: 0.7140 - accuracy: 0.7948 - val_loss: 1.4387 - val_accuracy: 0.6769 - lr: 1.0000e-04
Epoch 2/50
82/82 [==============================] - 9s 107ms/step - loss: 0.2352 - accuracy: 0.9238 - val_loss: 1.4491 - val_accuracy: 0.6662 - lr: 1.0000e-04
Epoch 3/50
82/82 [==============================] - 9s 107ms/step - loss: 0.1787 - accuracy: 0.9388 - val_loss: 0.3015 - val_accuracy: 0.8831 - lr: 1.0000e-04
Epoch 4/50
82/82 [==============================] - 9s 107ms/step - loss: 0.1888 - accuracy: 0.9400 - val_loss: 0.2436 - val_accuracy: 0.9277 - lr: 1.0000e-04
Epoch 5/50
82/82 [==============================] - 9s 107ms/step - loss: 0.0960 - accuracy: 0.9627 - val_loss: 0.2444 - val_accuracy: 0.9400 - lr: 1.0000e-04
Epoch 6/50
82/82 [==============================] - 9s 107ms/step - loss: 0.0592 - accuracy: 0.9811 - val_loss: 0.2984 - val_accuracy: 0.9277 - lr: 1.0000e-04
Epoch 7/50
82/82 [==============================] - 9s 1

...

82/82 [==============================] - 11s 117ms/step - loss: 0.7466 - accuracy: 0.7791 - val_loss: 1.2560 - val_accuracy: 0.6487 - lr: 1.0000e-04
Epoch 2/50
82/82 [==============================] - 9s 113ms/step - loss: 0.2459 - accuracy: 0.9173 - val_loss: 0.5108 - val_accuracy: 0.8228 - lr: 1.0000e-04
Epoch 3/50
82/82 [==============================] - 9s 109ms/step - loss: 0.1323 - accuracy: 0.9546 - val_loss: 0.2516 - val_accuracy: 0.9153 - lr: 1.0000e-04
Epoch 4/50
82/82 [==============================] - 9s 108ms/step - loss: 0.1414 - accuracy: 0.9565 - val_loss: 0.2847 - val_accuracy: 0.9199 - lr: 1.0000e-04
Epoch 5/50
82/82 [==============================] - 9s 107ms/step - loss: 0.0987 - accuracy: 0.9654 - val_loss: 0.3653 - val_accuracy: 0.8860 - lr: 1.0000e-04
Epoch 6/50
82/82 [==============================] - 9s 108ms/step - loss: 0.1062 - accuracy: 0.9669 - val_loss: 0.1632 - val_accuracy: 0.9492 - lr: 1.0000e-04
Epoch 7/50
82/82 [==============================] - 9s 1

2023-04-10 12:54:18.357491: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - ETA: 0s - loss: 0.6895 - accuracy: 0.7968

2023-04-10 12:54:27.510586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - 10s 118ms/step - loss: 0.6895 - accuracy: 0.7968 - val_loss: 1.4325 - val_accuracy: 0.6025 - lr: 1.0000e-04
Epoch 2/50
82/82 [==============================] - 9s 109ms/step - loss: 0.2807 - accuracy: 0.9115 - val_loss: 0.7360 - val_accuracy: 0.7350 - lr: 1.0000e-04
Epoch 3/50
82/82 [==============================] - 9s 108ms/step - loss: 0.2168 - accuracy: 0.9350 - val_loss: 0.4575 - val_accuracy: 0.8382 - lr: 1.0000e-04
Epoch 4/50
82/82 [==============================] - 9s 108ms/step - loss: 0.1476 - accuracy: 0.9481 - val_loss: 0.2836 - val_accuracy: 0.9106 - lr: 1.0000e-04
Epoch 5/50
82/82 [==============================] - 9s 107ms/step - loss: 0.1168 - accuracy: 0.9588 - val_loss: 0.3990 - val_accuracy: 0.8891 - lr: 1.0000e-04
Epoch 6/50
82/82 [==============================] - 9s 108ms/step - loss: 0.0536 - accuracy: 0.9781 - val_loss: 0.1729 - val_accuracy: 0.9430 - lr: 1.0000e-04
Epoch 7/50
82/82 [==============================] - 9s 1

In [12]:

def train_final_model(model_builder, epochs=20):
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    final_model = model_builder()

    # Definir el callback de EarlyStopping
    early_stopping = EarlyStopping(monitor='loss', patience=3, min_delta=0.001)

    # Agregar el callback al método fit
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

    callbacks = [early_stopping, reduce_lr]
    final_model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks=callbacks)

    return final_model

final_model = train_final_model(build_model,50)


Epoch 1/50


2023-04-10 12:59:28.319589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


102/102 [==============================] - 12s 107ms/step - loss: 0.6614 - accuracy: 0.8057 - lr: 1.0000e-04
Epoch 2/50
102/102 [==============================] - 10s 100ms/step - loss: 0.2542 - accuracy: 0.9209 - lr: 1.0000e-04
Epoch 3/50
102/102 [==============================] - 10s 100ms/step - loss: 0.1448 - accuracy: 0.9520 - lr: 1.0000e-04
Epoch 4/50
102/102 [==============================] - 10s 100ms/step - loss: 0.0906 - accuracy: 0.9667 - lr: 1.0000e-04
Epoch 5/50
102/102 [==============================] - 10s 100ms/step - loss: 0.0942 - accuracy: 0.9729 - lr: 1.0000e-04
Epoch 6/50
102/102 [==============================] - 10s 100ms/step - loss: 0.0497 - accuracy: 0.9849 - lr: 1.0000e-04
Epoch 7/50
102/102 [==============================] - 10s 100ms/step - loss: 0.0434 - accuracy: 0.9855 - lr: 1.0000e-04
Epoch 8/50
102/102 [==============================] - 10s 101ms/step - loss: 0.0487 - accuracy: 0.9809 - lr: 1.0000e-04
Epoch 9/50
102/102 [==============================]

In [14]:
test_loss, test_accuracy = final_model.evaluate(test_data)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Y_pred = final_model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

y_true = np.concatenate([y.numpy() for _, y in test_data.unbatch()])
y_true_labels = np.argmax(y_true.reshape(-1, len(class_names)), axis=1)

print('Classification Report:')
print(classification_report(y_true_labels, y_pred, target_names=class_names))

print('Confusion Matrix:')
print(confusion_matrix(y_true_labels, y_pred))


26/26 [==============================] - 1s 30ms/step - loss: 0.4783 - accuracy: 0.9128
Test Loss: 0.4783
Test Accuracy: 0.9128
26/26 [==============================] - 1s 27ms/step
Classification Report:
                                            precision    recall  f1-score   support

                         Grape___Black_rot       0.94      0.83      0.88       236
              Grape___Esca_(Black_Measles)       0.85      0.99      0.91       277
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)       1.00      0.87      0.93       216
                           Grape___healthy       0.90      1.00      0.95        85

                                  accuracy                           0.91       814
                                 macro avg       0.92      0.92      0.92       814
                              weighted avg       0.92      0.91      0.91       814

Confusion Matrix:
[[196  38   0   2]
 [  3 274   0   0]
 [  9  12 188   7]
 [  0   0   0  85]]
